In [1]:
import sys
print(sys.version)


3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]


In [3]:
%pip install dask ray


Note: you may need to restart the kernel to use updated packages.


In [4]:
import dask
import ray

print("Dask version:", dask.__version__)
print("Ray version:", ray.__version__)


2026-02-17 19:29:27,617	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Dask version: 2026.1.2
Ray version: 2.53.0


In [5]:
import dask.dataframe as dd
import ray

# Create sample log data (like real logs)
data = {
    "level": ["INFO", "ERROR", "INFO", "WARNING", "ERROR"],
    "message": ["Started", "Disk full", "User login", "High CPU", "Timeout"]
}

# Convert to Dask DataFrame
df = dd.from_pandas(__import__("pandas").DataFrame(data), npartitions=2)

# Dask test: count ERROR logs
error_count = df[df["level"] == "ERROR"].shape[0].compute()
print("Dask ERROR count:", error_count)

# Ray test: run anomaly detection-like task in parallel
ray.init(ignore_reinit_error=True)

@ray.remote
def detect_anomaly(log_level):
    if log_level == "ERROR":
        return "ANOMALY"
    return "NORMAL"

# Run Ray tasks in parallel
results = ray.get([detect_anomaly.remote(level) for level in data["level"]])

print("Ray classification results:", results)

ray.shutdown()


Dask ERROR count: 2


2026-02-17 19:37:15,611	INFO worker.py:2007 -- Started a local Ray instance.
F:\jyoshnitha\Lib\site-packages\ray\_private\worker.py:2046: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(


Ray classification results: ['NORMAL', 'ANOMALY', 'NORMAL', 'NORMAL', 'ANOMALY']
